# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:

- children — количество детей в семье

- days_employed — общий трудовой стаж в днях

- dob_years — возраст клиента в годах

- education — уровень образования клиента

- education_id — идентификатор уровня образования

- family_status — семейное положение

- family_status_id — идентификатор семейного положения

- gender — пол клиента

- income_type — тип занятости

- debt — имел ли задолженность по возврату кредитов

- total_income — ежемесячный доход

- purpose — цель получения кредита

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from collections import Counter



In [2]:
customer_table = pd.read_csv('/datasets/data.csv')


In [3]:
customer_table.head(15) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
customer_table.info() #распечатаем общую информацию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

1. На данном этапе смущают отрицательные значения в столбце days_employed. Возможно это просто ошибка и их можно перезаписать по модулю.

2. В столбцах, total_income и days_employed имеются отсутствующие значения. Можно рассмотреть вариант замены по среднему значению группы или удаление строк с отсутствующими значениями в целом, если она не повлияет на результаты исследования.

3. В столбце education одинаковые значения написаны разными регистрами, нужно привести к одному виду.

4. Значения в столбце days_employed и total_income можно привести к целочисленному типу, это удобно для вычислений и наглядности.

5. В столбце purpose содержатся похожие друг на друга цели, но выраженные разными словами.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
customer_table['days_employed'] = customer_table['days_employed'].abs() 
#возьмем значения по модулю
customer_table.head(15)  

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Значения заменились на положительные, теперь переходим к заполнению пропусков.

In [6]:
#заменим пропущенные значения days_employed средним значением по типам занятости
customer_table['days_employed'] = customer_table.groupby('income_type')['days_employed'].apply(lambda x: x.fillna(x.mean())) 
#заменим пропущенные значения total_income средним значением ежемесячного дохода по группам
customer_table['total_income'] = customer_table.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.mean()))


In [7]:
customer_table.info() #посмотрим изменилась ли общая информация в таблице и восстановились ли пропушенные значения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [8]:
customer_table.head(15) #выведем данные для проверки

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Отсутствующие значения в строках заполнены, теперь приведем столбец education в единый стиль.

In [9]:
# приведем значения к нижнему регистру 
customer_table['education'] = customer_table['education'].str.lower()
customer_table.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Значения в столбце days_employed сделали положительными.

В столбцах, total_income и days_employed заполнили отсутствующие значения по среднему значению столбца, соответствующему группе типа занятости. 

В столбце education значения привели к одному регистру.




### Замена типа данных

Используем функцию astype для замены типа данных, так как столбцы  имеют чистый формат float64 и  перевод в int будет безопасным

In [10]:
#из float в int
customer_table['days_employed'] = customer_table['days_employed'].astype('int')
customer_table['total_income'] = customer_table['total_income'].astype('int')

In [11]:
#проверим тип значений
customer_table.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

In [12]:
customer_table.head(10)   #распечатаем первые 10 строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Значения в столбце days_employed и total_income привели к целочисленному типу, так более наглядно и удобнее для вычислений.
Для замены типа данных был использован метод astype, так как мы уже обработали пропущенные значения (иначе можно было бы использовать to_numeric).

### Обработка дубликатов

In [13]:
#считаем количество дубликатов в данных
customer_table.duplicated().sum()

71

Дубликаты имеются. Скорее всего просто имеет место случайное копирование строк, типичное задвоение данных. 
Займемся их удалением.

In [14]:
#удаляем дубликаты 
customer_table = customer_table.drop_duplicates().reset_index(drop = True)
#проверяем еще раз количество
customer_table.duplicated().sum()

0

**Вывод**

Дубликаты удалены. 

### Лемматизация

В столбце, отражающем цель получения кредита нужно выделить цели, имеющие общий смысл и назвать их одинаково.
Для начала необходимо выяснить количество значений.

In [15]:
# Посчитаем все варианты целей кредита.
customer_table['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Среди всех возможных целей получения кредита можно выделить основные 5 целей, отражающие основные причины.
Можно сделать этот список вручную, так как количество выявленных причин небольшое.
После этого можно провести лемматизацию и заменитить значения на ключевые слова.


In [16]:
# создадим список основных целей кредита
categories = ['сдача', 'коммерческий', 'жилье', 'образование', 'свадьба', 'недвижимость', 'автомобиль']

# проведем лемматизацию,
# заменим полученный список лемм в каждой строке на главное ключевое слово с помощью создания функции
# из списка категорий, создаем новый столбец
m = Mystem()
def lemmatize(text):
    lemma = m.lemmatize(text)
    for element in categories:
        if element in lemma:
            lemma = element
    return lemma

customer_table['purpose_group'] = customer_table['purpose'].apply(lemmatize)        
customer_table.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


Новый столбец с ключевыми словами создан, посчитаем сколько получилось значений в нем.


In [17]:
#подсчет количества ключевых слов
customer_table['purpose_group'].value_counts()

недвижимость    5040
автомобиль      4306
образование     4013
жилье           3809
свадьба         2324
коммерческий    1311
сдача            651
Name: purpose_group, dtype: int64

Так как жилье это и есть  недвижимость, то сделаем соответствующую замену. Можно еще заменить категории по покупке коммерческой недвижимости или покупке недвижимости для сдачи  на инвестиционную цель.

In [18]:
customer_table.loc[customer_table['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'
customer_table.loc[customer_table['purpose_group'] == 'коммерческий', 'purpose_group'] = 'инвестиция'
customer_table.loc[customer_table['purpose_group'] == 'сдача', 'purpose_group'] = 'инвестиция'

In [19]:
customer_table['purpose_group'].value_counts()

недвижимость    8849
автомобиль      4306
образование     4013
свадьба         2324
инвестиция      1962
Name: purpose_group, dtype: int64

In [20]:
customer_table.head(10) #распечатаем, чтобы перепроверить

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


**Вывод**

Из цели получения кредита каждого клиента было выделено по ключевому слову.
Подобный способ выделения целей упрощает дальнейший анализ, при этом сохраняя смысл цели получения кредита.

Цели, содержащие производные слова от слова жилье были заменены на категорию недвижимость.
Так как приобретение коммерческой недвижимости или недвижимости для сдачи относится к инвестиционным целям, и на этих вариантах можно увеличить капитал, мы выделяем это в отдельную категорию, а не объединяем с недвижимостью.

### Категоризация данных

По целям кредита мы уже присвоили категории, по семейному статусу они уже сгруппированы. 
Выделим категории по уровню дохода,  по количеству детей, так как выделение именно этих категорий необходимо нам для ответа на интересующие нас вопросы относительно цели исследования.


Разобьем на категории по доходу total_income:

Бедный x <= 50000;

Средний 50000 < x <= 150000;

Выше среднего 150000 < x < 200000;

Богатый x >= 200000;

In [21]:
#фунция, присваивающая выделенные категории

def income_group(row):
    if row['total_income'] <= 50000:
        return 'бедный'
    elif 50000 < row['total_income'] <= 100000:
        return 'средний'
    elif 100000 < row['total_income'] < 200000:
        return 'выше среднего'
    else:
        return 'богатый'

# применим функцию к столбцу доходов
customer_table['income_group'] = customer_table.apply(income_group, axis=1)
#вывод таблицы
customer_table.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,богатый
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,выше среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,богатый
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,выше среднего
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,богатый
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,богатый
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,выше среднего
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,средний
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,выше среднего


Сгруппируем по количеству детей.

In [22]:
# напишем функцию для категоризации клиентов: 1 — есть дети, 0 — нет детей.
def determine_children(children):
    if children > 0: return 1
    else: return 0

# добавим новый столбец в таблицу
customer_table['child_exist'] = customer_table['children'].apply(determine_children)
customer_table.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,income_group,child_exist
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,богатый,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,выше среднего,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,выше среднего,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,богатый,1
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,выше среднего,0


**Вывод**

Категоризация по целям.

Чаще всего клиенты берут кредит для приобретения недвижимости для личного пользования.

In [23]:
customer_table['purpose_group'].value_counts()

недвижимость    8849
автомобиль      4306
образование     4013
свадьба         2324
инвестиция      1962
Name: purpose_group, dtype: int64

Категоризация по доходам.

Большая часть, запрашивающих кредит имеет доход выше среднего. 

In [24]:
customer_table['income_group'].value_counts()

выше среднего    11423
богатый           5568
средний           4091
бедный             372
Name: income_group, dtype: int64

Категоризация по количеству детей.

Большинство, подавших запрос на получение кредита не имеют детей.

In [25]:
customer_table['child_exist'].value_counts()

0    14138
1     7316
Name: child_exist, dtype: int64

Категоризация по семейному положению.

Большинство кредитов выдается людям, состояшим в официальном браке.

In [26]:
customer_table['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Мы категорировали данные, без которых невозможно было бы провести дальнейший анализ для ответов на поставленные в исследовании вопросы. 
Можно приступить к дальнейшему анализу уже на основе сформированных категорий.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [27]:
# построим сводную таблицу
child_table_pivot = customer_table.pivot_table(index = ['child_exist'], values = 'debt')

child_table_pivot.head()

,debt
child_exist,
0,0.075258
1,0.092537


**Вывод**

Судя по  сводной таблице, менее склонны к невыплатам кредита клиенты, не имеющие детей. 
Это можно объяснить тем, что в сложном материальном положении в семьях с детьми приоритетнее выступает вопрос обеспечения семьи необходимыми средствами существования, задолженность по кредиту уходит на задний план.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [28]:
#построим сводную таблицу по задолженности и семейному статусу
family_pivot = customer_table.pivot_table(index = ['family_status'],  values = 'debt')
family_pivot.head(10)


,debt
family_status,
Не женат / не замужем,0.097509
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452


**Вывод**

Клиенты, никогда не состоящие в официальном браке, более склонны к просрочке по выплатам кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [29]:
#построим сводную таблицу по задолженности и уровню дохода
income_pivot = customer_table.pivot_table(index = ['income_group'], values = 'debt')
income_pivot

,debt
income_group,
бедный,0.061828
богатый,0.069684
выше среднего,0.087455
средний,0.080909


**Вывод**

Факт того, что низкий уровень задолженностей наблюдается у людей, имеющих наивысший доход понятен. У этой категории практически не возникает проблем с финансами. Вместе с тем удивляет тот факт, что люди с наименьшим уровнем дохода реже оказываются в должниках. Это можно объяснить только тем, что люди с низким доходом более ответственно подходят к распределению расходов.

- Как разные цели кредита влияют на его возврат в срок?

In [30]:
#построим сводную таблицу по задолженности и целям кредита
purpose_pivot = customer_table.pivot_table(index = ['purpose_group'], values = 'debt')
purpose_pivot

,debt
purpose_group,
автомобиль,0.093590
инвестиция,0.076962
недвижимость,0.071307
образование,0.092200
свадьба,0.080034


**Вывод**

Заемщики, которые берут кредит для приобретения недвижимости, проявляют себя более ответственно и реже имеют задолженности.
Неожиданный факт: клиенты, берущие кредит на свадьбу, не самые безответственные.

## Шаг 4. Общий вывод

Для описания идеального  клиента интересно было бы в будущем дополнительно посмотреть на зависимость от пола. 
Но исходя из данных,  по которым мы провели анализ можно сказать следующее.
Идеальный портрет клиента - это человек, находящийся в официальном браке, без детей, с доходом более 200000 рублей в месяц, желающий с помощью кредитных средств приобрести собственное жилье.
